<a href="https://colab.research.google.com/github/katiacardoso/EMAP_Chatbot/blob/main/chatbot_emap_bert_base_principal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação: Chatbot com Bert Base

Importação de bibliotecas

In [7]:
!pip install transformers

In [8]:
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

import pandas as pd
import json

import requests
from io import StringIO

print(torch.__version__)

import sys
from transformers import __version__ as transformers_version
import transformers as transformers

print("Versão do Python", sys.version)
print("Versão do json:", json.__version__)
print("Versão do requests:", requests.__version__)
print("Versão do numpy:", np.__version__)
print("Versão do pandas:", pd.__version__)
print("Versão do torch:", torch.__version__)
print("Versão do transformers:", transformers.__version__)

2.1.0+cu118
Versão do Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Versão do json: 2.0.9
Versão do requests: 2.31.0
Versão do numpy: 1.23.5
Versão do pandas: 1.5.3
Versão do torch: 2.1.0+cu118
Versão do transformers: 4.35.0


Parte do código que é responsável por carregar um modelo BERT pré-treinado e seu tokenizador associado. O modelo BERT será usado para responder a perguntas com base em um contexto fornecido. Essa configuração permite que o chatbot compreenda e responda às perguntas dos usuários com base no conhecimento prévio adquirido durante o treinamento do modelo BERT.

- O tokenizador é usado para dividir o texto em unidades menores chamadas tokens, o que é necessário para processar texto com modelos de linguagem como o BERT





In [9]:
#importação dos dados
url = 'https://raw.githubusercontent.com/katiacardoso/EMAP_Chatbot/main/perguntas_respostass.csv'

# Baixar o conteúdo do arquivo usando requests
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Criar um objeto pandas DataFrame a partir do conteúdo baixado
    df = pd.read_csv(StringIO(response.text), usecols=['Pergunta', 'Resposta'])
    # Agora você pode usar o DataFrame 'data' normalmente
else:
    print("Falha ao baixar o arquivo. Código de status:", response.status_code)


df.to_csv('perguntas_respostass.csv', index=False)

In [10]:

# Carregue o modelo BERT pré-treinado e o tokenizador
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)



In [11]:

# Carregue o arquivo CSV com perguntas e respostas
#df = pd.read_csv("perguntas_respostass.csv")

# Converter os dados para um formato adequado para JSON
data = []
for index, row in df.iterrows():
    pergunta = row['Pergunta']
    resposta = row['Resposta']
    data.append({'pergunta': pergunta, 'resposta': resposta})


# Salvar os dados como JSON em um arquivo
with open('perguntas_respostas.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

# Carregar dados do arquivo JSON
with open('perguntas_respostas.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extrair perguntas e respostas
perguntas = [qa['pergunta'] for qa in data]
respostas = [qa['resposta'] for qa in data]

Na **Tokenização da Pergunta do Usuário** é feita uma conversão da pergunta em uma sequência de ID's de tokens que o modelo BERT pode entender.



In [12]:
# Função para vetorizar texto usando BERT
def vetorizar_texto(texto):
    inputs = tokenizer(texto, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Média das embeddings das palavras
    return embeddings

# Vetorizar perguntas usando BERT
perguntas_embeddings = torch.cat([vetorizar_texto(pergunta) for pergunta in perguntas])

In [13]:
# Função para encontrar a melhor resposta com base na pergunta do usuário
def chatbot(user_input):
    user_input_embedding = vetorizar_texto(user_input)
    similarity_scores = cosine_similarity(user_input_embedding, perguntas_embeddings)
    best_match_index = similarity_scores.argmax()
    resposta = respostas[best_match_index]
    return resposta

a lista **conversa** é usada para armazenar os pares de pergunta e resposta em forma de dicionários. Cada vez que o usuário faz uma pergunta e o chatbot responde, um novo dicionário é adicionado à lista.

Ele só salva após digitar para Sair, é necessário ir salvando de acordo entra uma nova pergunta e gera uma nova resposta?
R: só ir alterando o paramêtro: "a" para ir adicionando e salvando e se for "w" é apenas escrita e a cada compilação some


In [14]:
# Lista para armazenar as perguntas e respostas
conversa = []

# Inicie uma conversa com o chatbot
while True:
    user_input = input("Você: ")
    if user_input.lower() == "sair":
        print("Chatbot: Adeus! Até logo.")
        break
    response = chatbot(user_input)
    print("Chatbot:", response)

    # Armazene a pergunta do usuário e a resposta do chatbot na lista
    conversa.append({'pergunta': user_input, 'resposta': response})

    # Salve a lista como JSON em um arquivo
    with open('conversa.json', 'a', encoding='utf-8') as json_file:
        json.dump(conversa, json_file, ensure_ascii=False, indent=4)



Você: Como chegar no Porto?
Chatbot: Confira aqui o mapa de localização e seja bem-vindo! http://www.portodoitaqui.ma.gov.br/porto-do-itaqui/como-chegar 
Você: Como chegar ao Porto?
Chatbot: Confira aqui o mapa de localização e seja bem-vindo! http://www.portodoitaqui.ma.gov.br/porto-do-itaqui/como-chegar 
Você: Capitu traiu Bentinho?
Chatbot: Na EMAP, empresa pública que administra o Porto do Itaqui, as contratações são feitas por meio de concurso público. Existem várias outras empresas privadas que atuam no porto organizado (saiba quais órgãos e empresas atuam dentro da área do Porto do Itaqui em: http://www.portodoitaqui.ma.gov.br/porto-do-itaqui/comunidade-portuaria ). Sobre o nosso último concurso público, você pode conferir por aqui: http://www.portodoitaqui.ma.gov.br/emap/recursos-humanos/concurso-publico.
Você: Sair
Chatbot: Adeus! Até logo.
